Import Stuff

In [267]:
import plotly
plotly.__version__

import plotly.plotly as py
import plotly.graph_objs as go
import geopy.distance
import os  

from math import cos, asin, sqrt, radians
import csv  
from pathlib import Path  
from timeit import default_timer as timer  
from datetime import datetime
from pulp import *

from math import sin, cos, sqrt, atan2, radians
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
import numpy as np
import pandas as pd

from itertools import product

df_demand_base = pd.read_csv('Demand.csv')
df_demand_base['Demand']=30
df_supply = pd.read_csv('Supply.csv')
df_supply['Exist']=1
df_potential = pd.read_csv('PostalCode.csv')
df_potential=df_potential.rename(columns = {'Latitude':'latitude','Longitude':'longitude','PostalCode':'Name'})
df_local_potential = df_potential[(df_potential.longitude<=max(df_demand_base['longitude'])) & \
    (df_potential.longitude>=min(df_demand_base['longitude']))& \
    (df_potential.latitude<=max(df_demand_base['latitude'])) & \
    (df_potential.latitude>=min(df_demand_base['latitude']))]
df_local_potential=df_local_potential[['Name','longitude','latitude']].sample(50)

mapbox_access_token = 'pk.eyJ1IjoibWFrYWxhbjYwNCIsImEiOiJjanUzajlram8wbnNoNDNzYXFqOGViZ2k1In0.jXYO-ZgW33bqv4c6-vK12g'

import plotly 
# plotly.tools.set_credentials_file(username='makalan604', api_key='gbwrf6T8Xs7WJ6FiJ5Pq')
plotly.tools.set_credentials_file(username='alanmakoutlook', api_key='I6cjehQZ4V3G4bDFUZGm')


Sandbox

In [378]:
old=20
tots=55
demand, supply, tot_travel  = Optimize(df_demand_base,df_supply,df_local_potential,old,tots,100)
plottingmap(demand,supply,'Old Hub'+str(old)+'   Total Hub'+str(tots),1)
tot_travel

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Optimal


14331.691008988715

In [381]:
set(demand.SupplyID)


{1,
 2,
 3,
 4,
 6,
 7,
 8,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 19,
 20,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 60,
 63,
 87}

In [382]:
# plottingmap(df_demand_base,df_supply,'Original')
expected_capacity=100

Total_HUBS=30

All_Anal=pd.DataFrame(np.zeros((len(df_supply),Total_HUBS)))



for j in range(0,Total_HUBS,5):
    for i in range(0,len(df_supply),2):
        num_keep=len(df_supply)-i
        total_hubs=j
        demand, supply, tot_travel  = Optimize(df_demand_base,df_supply,df_local_potential,num_keep,total_hubs,expected_capacity)
#         if tot_travel<0:  
            #     plottingmap(demand,supply,'Old Hub'+str(num_keep)+'   Total Hub'+str(total_hubs))
        All_Anal[total_hubs][num_keep]=int(tot_travel)
        
        print('Old Hub'+str(num_keep)+'   Total Hub'+str(total_hubs+1)+"  distance:"+str(int(tot_travel)))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Infeasible
Old Hub47   Total Hub1  distance:-1
Infeasible
Old Hub45   Total Hub1  distance:-1
Infeasible
Old Hub43   Total Hub1  distance:-1
Infeasible
Old Hub41   Total Hub1  distance:-1
Infeasible
Old Hub39   Total Hub1  distance:-1
Infeasible
Old Hub37   Total Hub1  distance:-1
Infeasible
Old Hub35   Total Hub1  distance:-1
Infeasible
Old Hub33   Total Hub1  distance:-1
Infeasible
Old Hub31   Total Hub1  distance:-1
Infeasible
Old Hub29   Total Hub1  distance:-1
Infeasible
Old Hub27   Total Hub1  distance:-1
Infeasible
Old Hub25   Total Hub1  distance:-1
Infeasible
Old Hub23   Total Hub1  distance:-1
Infeasible
Old Hub21   Total Hub1  distance:-1
Infeasible
Old Hub19   Total Hub1  distance:-1
Infeasible
Old Hub17   Total Hub1  distance:-1
Infeasible
Old Hub15   Total Hub1  distance:-1
Infeasible
Old Hub13   Total Hub1  distance:-1
Infeasible
Old Hub11   Total Hub1  distance:-1
Infeasible
Old Hub9   Total Hub1  distance:-1
Infeasible
Old Hub7   Total Hub1  distance:-1
Infeasible
Old 

Functions

In [12]:
def closest(lat1, lon1, lat2, lon2):  
#     lat1 = radians(nlat1)
#     lon1 = radians(nlon1)
#     lat2 = radians(nlat2)
#     lon2 = radians(nlon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1


    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    km = 6367 * c
    least=np.argmin(km)
    return [least+1 ,km[least]]

def distance(lat1, lon1, lat2, lon2):  
#     lat1 = radians(nlat1)
#     lon1 = radians(nlon1)
#     lat2 = radians(nlat2)
#     lon2 = radians(nlon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1


    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    km = 6367 * c
    
    return km

In [371]:
def Optimize(df_demand_f,df_supply_f,df_potential,oldhubs,totshub,expected_capacity):


    Num_demand=len(df_demand_f)
    df_potential=df_local_potential


    demand=df_demand_f.Demand

    df_all_location=df_supply_f
    df_all_location['Exist']=1
    df_all_location.head()
    df_all_location=df_all_location.append(df_potential)

    df_all_location['Capacity'].fillna(expected_capacity, inplace=True)
    df_all_location['Exist'].fillna(0, inplace=True)
    df_all_location['New']=np.absolute(1-df_all_location['Exist'])
    df_all_location=df_all_location.reset_index()


    df_all_location=df_all_location.fillna(method='ffill')
    temp=0
    for i in range(len(df_all_location)):
        if df_all_location.SupplyID[i]<=temp:
            df_all_location.SupplyID[i]=temp+1
            temp=df_all_location.SupplyID[i]
        else:
            temp=df_all_location.SupplyID[i]


    existing=df_all_location['Exist']
    capacity = df_all_location.Capacity
    Num_supply=len(df_all_location)


    cost= np.random.rand(len(df_demand_f),len(df_all_location))

    for i in range(len(df_demand_f)):
        cost[i,:]=distance(df_demand_f['latitude'][i]  ,df_demand_f['longitude'][i]   ,df_all_location['latitude']  ,df_all_location['longitude'])

    prob= LpProblem("Transportation",LpMinimize)

    route_select = LpVariable.dicts("Route", product(range(Num_demand), range(len(df_all_location))), None,cat='Binary')

    hub_select = LpVariable.dicts("Hub", range(len(df_all_location)), None,cat='Binary')

    # LpVariable.dicts("Route",(Warehouses,Bars),0,None,LpInteger)

    # min cost
    prob += (lpSum((cost[l, c] * route_select[l, c] ) for l in range(Num_demand) for c in range(len(df_all_location))))+(lpSum(hub_select[c]  for c in range(len(df_all_location))))

    prob += lpSum(hub_select[c] for c in range(len(df_supply_f))) >= num_keep

    prob += lpSum(hub_select[c] for c in range(len(df_all_location))) == total_hubs



    for l in range(Num_demand):
        prob += lpSum(demand[l] * route_select[l, c] for c in range(Num_supply)) >= demand[l] , "All_Demand_is_matched_%s"%l

    for c in range(len(df_all_location)):
        contraint = lpSum(demand[l] * route_select[l, c] for l in range(Num_demand)) <= capacity[c]*hub_select[c] 
        prob += contraint, "Capacity_not_breached_%s"%c

    # for c in range(len(df_all_location)):
    #     prob += lpSum(demand[l] * route_select[l, c] for l in range(Num_demand)) <= capacity[c] , "Capacity_not_breached_%s"%c



    # prob += lpSum((route_select[l, c] for l in range(Num_demand))  for c in range(len(df_all_location))) <= total_hubs

    # prob += numpy.count(route_select[l,c] for l in range(Num_demand) for c in range(len(df_all_location)))) >= num_keep
    # count = sum((l,c), route_select[l,c])
    # prob += lpSum((sum((df_all_location['Exist'][c] * route_select[l, c] )for l in range(Num_demand))>=0) for c in range(len(df_all_location))) >= num_keep




    prob.solve()  # Solve with the default solver
    print(pulp.LpStatus[prob.status])
    #       # Print the solution status

    varsdict = {}
    for v in prob.variables():
        varsdict[v.name] = v.varValue



    All=pd.DataFrame(list(varsdict.items()), columns=['RouteName', 'Travelled'])

    import re
    i=1
    Prioritized=All[len(df_all_location):].reset_index(drop=True)
    temp1= np.random.rand(len(Prioritized))
    temp2= np.random.rand(len(Prioritized))
    for i in range(len(Prioritized)):
        result = re.findall(r'\d+',Prioritized.RouteName[i])
        temp1[i]=int(result[0])
        temp2[i]=int(result[1])

    Prioritized['SupplyID']=temp2+1
    Prioritized['DemandID']=temp1+1

    togethered=pd.merge(df_demand_f,Prioritized[Prioritized.Travelled==1][['SupplyID','DemandID']],on='DemandID', how='left')
    togethered['SupplyID']=togethered['SupplyID_y']
    df_demand_opx_capacity=togethered.fillna(0).drop(labels=['SupplyID_y','SupplyID_x'],axis=1)
    df_demand_opx_capacity.SupplyID = df_demand_opx_capacity.SupplyID.astype(int)
    if pulp.LpStatus[prob.status] =='Optimal':
        total_distance=pulp.value(prob.objective)
    else:
        total_distance=-1
        
    return df_demand_opx_capacity,df_all_location,total_distance

In [302]:
def plottingmap(df_demand,df_supply,name,selected):
    init_notebook_mode(connected=True)
    df_supply=pd.merge(df_supply ,pd.DataFrame(df_demand.groupby("SupplyID")['Demand'].sum()),how='left',on='SupplyID').fillna(value=0)
    df_supply['Demand']=df_supply['Demand'].astype(int)
    
    df_demand_combined = pd.merge(df_demand,df_supply, on='SupplyID')

    Demand_text='Demand:'+df_demand['DemandID'].astype(str)+','+df_demand['Name']+','+df_demand['SupplyID'].astype(str)
    Supply_text= 'Supply:'+df_supply['SupplyID'].astype(str)+','+df_supply['Demand'].astype(int).astype(str)+'out of'+df_supply['Capacity'].astype(str)+','+df_supply['Name']
    Line_text=df_demand_combined['DemandID'].astype(str)+','+df_demand_combined['SupplyID'].astype(str)


    
    
    demand = [
        go.Scattermapbox(
            lon = df_demand['longitude'],
            lat = df_demand['latitude'],
            text=Demand_text,
            hoverinfo = 'text',
            name='Demand',
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=5,
                color='rgb(242, 0, 0)',
                opacity=0.7
            ),
        )
    ]

    supply_old = [
        go.Scattermapbox(
            lon = df_supply[df_supply.Exist==1]['longitude'],
            lat = df_supply[df_supply.Exist==1]['latitude'],
            text='old',
            hoverinfo = 'text',
            name='Supply',
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=5,
                color='rgb(0, 200, 200)',
                opacity=0.7
            ),

        )
    ]
    
    if selected:
        
        supply_new= [
            go.Scattermapbox(
                lon = df_supply[(df_supply.Exist==0)]['longitude'],
                lat = df_supply[(df_supply.Exist==0)]['latitude'],
    #             text=Supply_text,
                text='new',
                hoverinfo = 'text',
                name='Supply',
                mode='markers',
                marker=go.scattermapbox.Marker(
                    size=5,
                    color='rgb(0, 0, 200)',
                    opacity=0.7
                ),

            )
        ]
    else:
                supply_new= [
            go.Scattermapbox(
                lon = df_supply[(df_supply.Exist==0)&(df_supply.Demand>0)]['longitude'],
                lat = df_supply[(df_supply.Exist==0)&(df_supply.Demand>0)]['latitude'],
    #             text=Supply_text,
                text='new',
                hoverinfo = 'text',
                name='Supply',
                mode='markers',
                marker=go.scattermapbox.Marker(
                    size=5,
                    color='rgb(0, 0, 200)',
                    opacity=0.7
                ),

            )
        ]


    N= len(df_demand_combined)
    c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]

    delivery_paths = []
    for i in range(len(df_demand_combined)):
        delivery_paths.append(
            go.Scattermapbox(

                lon = [df_demand_combined['longitude_x'][i], df_demand_combined['longitude_y'][i]],
                lat = [df_demand_combined['latitude_x'][i], df_demand_combined['latitude_y'][i]],
                text=Line_text,
                hoverinfo = 'none',
                name='Routes',
                mode = 'lines',
                    line = go.scattermapbox.Line(
                    width = 1,
#                     color = c[df_demand_combined.SupplyID[i]],

                ),
    #                opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
            )
        )



    layout = go.Layout(
        autosize=True,
        hovermode='closest',
        showlegend = False,
        mapbox=go.layout.Mapbox(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=51.05,
                lon=-114.07
            ),
            pitch=0,
            zoom=10
        ),
    )

    fig = go.Figure(data=delivery_paths+supply_old+supply_new+demand, layout=layout)

    
    return(py.iplot(fig, filename=name))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Old Hub10   Total Hub1  distance:-1
Old Hub8   Total Hub1  distance:-1
Old Hub6   Total Hub1  distance:-1
Old Hub4   Total Hub1  distance:-1
Old Hub2   Total Hub1  distance:-1
Old Hub10   Total Hub6  distance:-1
Old Hub8   Total Hub6  distance:-1
Old Hub6   Total Hub6  distance:-1
Old Hub4   Total Hub6  distance:-1
Old Hub2   Total Hub6  distance:-1
Old Hub10   Total Hub11  distance:3256
Old Hub8   Total Hub11  distance:3256
Old Hub6   Total Hub11  distance:3256
Old Hub4   Total Hub11  distance:3260
Old Hub2   Total Hub11  distance:3324
Old Hub10   Total Hub16  distance:3237
Old Hub8   Total Hub16  distance:3237
Old Hub6   Total Hub16  distance:3237
Old Hub4   Total Hub16  distance:3253
Old Hub2   Total Hub16  distance:3324
Old Hub10   Total Hub21  distance:3237
Old Hub8   Total Hub21  distance:3237
Old Hub6   Total Hub21  distance:3237
Old Hub4   Total Hub21  distance:3253
Old Hub2   Total Hub21  distance:3324
Old Hub10   Total Hub26  distance:3237
Old Hub8   Total Hub26  distance:323

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Optimal


14331.691008988715

In [329]:
All=pd.DataFrame(list(varsdict.items()), columns=['RouteName', 'Travelled'])

import re
i=1
Prioritized=All[len(df_all_location):].reset_index(drop=True)
temp=np.zeros((len(df_all_location),(Num_demand)))
temp1= np.random.rand(len(Prioritized))
temp2= np.random.rand(len(Prioritized))
for i in range(len(Prioritized)):
    result = re.findall(r'\d+',Prioritized.RouteName[i])
    temp[int(result[1]),int(result[0])]=Prioritized.Travelled[i]


In [367]:
(set(df_demand_opx_capacity.SupplyID))

{1,
 2,
 3,
 4,
 6,
 7,
 8,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 19,
 20,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 60,
 63,
 87}